<a href="https://colab.research.google.com/github/AyonabhChakraborty/SentimentalAnalysis/blob/main/TwitterAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Description : This is a sentiment analysis program


In [1]:
#Importing libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [ ]:
#Load the data
from google.colab import files
uploaded = files.upload()

In [ ]:
#Getting the data
log = pd.read_csv('Login.csv')

In [ ]:
#Twitter API credentials
consumerKey = log['key'][0]
consumerSecret = log['key'][1]
accessToken = log['key'][2]
accessTokenSecret = log['key'][3]



In [ ]:
#Creating the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

#Setting the access token and access token secret
authenticate.set_access_token(accessToken)

#Create the API object while passing in the auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [ ]:
#Extraction of Tweets from twitter user
posts = api.user_timeline(screen_name = "ElonMusk", count = 1000, lang = 'en', tweet_mode="extended")


#Printing the last few tweets from the account
print("Show the 5 recent tweets : \n")
i=1
for tweet in posts[0:5]:
    print(str(i) + ')' + tweet.full_text + '\n')
    i = i + 1


In [ ]:
#Creation of dataframe with a column for Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts] , columns = ['Tweets'])

#Show the first 5 rows of data
df.head()


In [ ]:
#Cleaning the texts
#Function Creation
def cleanText(text):
  text = re.sub('@[A-Za-z0-9]+','',text ) #Removing the tags and mentions
  text = re.sub(r'#','',text) #Removing Hashtags
  text = re.sub(r'RT[\s]+', '', text) #Removing the RT symbol
  text = re.sub(r'https?://\\S+', '', text) #Removing any present hyperlinks


  return text


df['Tweets'] = df['Tweets'].apply(cleanText)


#Show the cleaned text
df


In [ ]:
#Creation of Subjectivity Function
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity


#Create a function to get the polarity
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)


df

In [ ]:
#Plotting the Word Cloud
allWords = ''.join(twts for twts in df['Tweets'])
wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allWords)

plt.imshow(wordCloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()


In [ ]:
#Creating function for positive, negative and neutral analysis
def getAnalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else :
    return 'Positive'

df['Analysis'] = df['Polarity'].apply(getAnalysis)

#Display the dataframe
df


In [ ]:
#Printing all the positive tweets
j=1
sortedDF = df.sort_values(by=['Polarity'])
for i in range(0, sortedDF.shape[0]):
    if sortedDF['Analysis'][i] == 'Positive'):
        print(str(j) + ')' + sortedDF['Tweets'][i])
        print()
        j = j+1


        


In [ ]:
#print the negative tweets
p=1
sortedDFN = df.sort_values(by=[Polarity])
for a in range(0, sortedDFN.shape[0]):
    if(sortedDFN['Analysis'][a] == 'Negative'):
      print(str(p) + ')' + sortedDFN['Tweets'][i])
      print()
      p=p+1               
                                           

In [ ]:
#plotting the polarity and the subjectivity
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
  plt.scatter(df['Polarity'][i], df['Subjectivity'][i], color = 'Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

